In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import yaml
from ultralytics import YOLO


In [2]:
with open("data.yaml") as file:
    data = yaml.safe_load(file)

In [3]:
image_paths = data["train"]
annotation_paths = data["train_annotations"]
class_names = data["names"]
output_dir = "bb_images/"

In [5]:
def draw_bounding_box(image_path,annotation_path,output_path):
    image = cv2.imread(image_path)
    h, w, _ = image.shape
    with open(annotation_path, 'r') as f:
        annotations = f.readlines()

    for ann in annotations:
        ann = ann.strip().split()
        class_id = int(ann[0])
        if class_id < 0 or class_id >= 14:
            print(f"Invalid class_id {class_id} in file {annotation_path}")
            continue
        x_center = float(ann[1]) * w
        y_center = float(ann[2]) * h
        bbox_width = float(ann[3]) * w
        bbox_height = float(ann[4]) * h
        x1 = int(x_center - bbox_width / 2)
        y1 = int(y_center - bbox_height / 2)
        x2 = int(x_center + bbox_width / 2)
        y2 = int(y_center + bbox_height / 2)
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, class_names[class_id], (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    cv2.imwrite(output_path, image)

In [6]:
for image_dir, annotation_dir in zip(image_paths, annotation_paths):
    for annotation_file in os.listdir(annotation_dir):
        if annotation_file.endswith('.txt'):
            image_file_base = annotation_file.replace('.txt', '')  # Get base name without extension
            possible_extensions = ['.jpg', '.jpeg']
            image_path = None
            
            for ext in possible_extensions:
                if os.path.exists(os.path.join(image_dir, image_file_base + ext)):
                    image_path = os.path.join(image_dir, image_file_base + ext)
                    break
            
            if image_path:
                annotation_path = os.path.join(annotation_dir, annotation_file)
                image_filename = os.path.basename(image_path)
                output_path = os.path.join(output_dir, image_filename)
                
                draw_bounding_box(image_path, annotation_path, output_path)
            else:
                print(f"Image file for annotation {annotation_file} not found in {image_dir}")

print("Processing completed.")

Image file for annotation train.txt not found in D:\juvdv2-vdvwc-1\Train\Sunny\Day
Image file for annotation train.txt not found in D:\juvdv2-vdvwc-1\Train\Sunny\Night
Processing completed.


In [12]:
#build a YOLOv9c model from scratch
model = YOLO("yolov9c.yaml")
#Display model information
model.info()

results = model.train(data="data.yaml", epochs = 20, imgsz = 224)

YOLOv9c summary: 618 layers, 25590912 parameters, 25590896 gradients, 104.0 GFLOPs
Ultralytics YOLOv8.2.48  Python-3.11.9 torch-2.3.1+cpu CPU (12th Gen Intel Core(TM) i5-12450HX)
engine\trainer: task=detect, mode=train, model=yolov9c.yaml, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=Fa

train: Scanning D:\juvdv2-vdvwc-1\Train\Rainny\Day.cache... 0 images, 2600 backgrounds, 0 corrupt: 100%|██████████| 2600/2600 [00:00<?, ?it/s]

WARNING  No labels found in D:\juvdv2-vdvwc-1\Train\Rainny\Day.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.



val: Scanning D:\juvdv2-vdvwc-1\Val\Rainny\Day.cache... 0 images, 200 backgrounds, 0 corrupt: 100%|██████████| 200/200 [00:00<?, ?it/s]

WARNING  No labels found in D:\juvdv2-vdvwc-1\Val\Rainny\Day.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


Plotting labels to runs\detect\train4\labels.jpg... 
zero-size array to reduction operation maximum which has no identity
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000556, momentum=0.9) with parameter groups 154 weight(decay=0.0), 161 weight(decay=0.0005), 160 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs\detect\train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G          0      12.84          0          0        224:  54%|█████▍    | 88/163 [15:51<13:31, 10.81s/it]


KeyboardInterrupt: 